In [ ]:
import matplotlib.pyplot as plt
from datasets.PupilCoreDatasetPupil import PupilCoreDataset
import torch

from torchvision.models.segmentation import FCN_ResNet50_Weights, fcn_resnet50
import numpy as np
import cv2
import utils
from models.ifOpened import ifOpenedModel
from models.trainers import PupilSegmentationTrainer, IfOpenedTrainer

In [ ]:
DATASET_LEN_TO_USE = 5000
dataset = PupilCoreDataset(
    "datasets/PupilCoreDataset/video5_eye0_video.avi",
    'datasets/PupilCoreDataset/video5_eye0_pupildata.csv',
    "datasets/PupilCoreDataset/video5_eye1_video.avi",
    'datasets/PupilCoreDataset/video5_eye1_pupildata.csv',
    DATASET_LEN_TO_USE
)
# dataset.get_pupil_ellipse()
# dataset.save_masks("datasets/PupilCoreDataset/created_masks")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
ifOpenedModel = ifOpenedModel()
PupilSegmentationModel = fcn_resnet50(weights=None, num_classes=1)

In [ ]:
dataset.load_masks("datasets/PupilCoreDataset/created_masks/eye0", "datasets/PupilCoreDataset/created_masks/eye1")

In [ ]:
torch.set_printoptions(edgeitems=100)
# for i in range(1):
#     print(dataset.eye0_masks[0].shape)
    # print(dataset[0])
#     plt.imshow(dataset.eye0_masks[i*50],cmap='gray')
#     plt.show()
#     plt.imshow(dataset.eye0_frames[i*50])
#     plt.show()

In [ ]:
if_opened_trainer = IfOpenedTrainer(model=ifOpenedModel, dataset=dataset, dataset_len=DATASET_LEN_TO_USE)
pupil_trainer = PupilSegmentationTrainer(model=PupilSegmentationModel, dataset=dataset, dataset_len=DATASET_LEN_TO_USE)

In [ ]:
# dataset_pupil.get_pupil_ellipse()

In [ ]:
# def visualize(model, dataloader, device, num_images: int = 5):
#     model.eval()
#     fig = plt.figure()
#     with torch.no_grad():
#         for i, (inputs, labels) in enumerate(dataloader):
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             outputs = model(inputs)
            
#             center_out = list(map(int, outputs[0].cpu().numpy()))
#             center_labels = list(map(int, labels[0].cpu().numpy()))
#             print(f"center_out: {center_out}", f"center_label: {center_labels}")
#             image = np.transpose(inputs[0].cpu().numpy(), (1, 2, 0)).copy()
            
#             # fig.add_subplot(1, num_images, i+1)
#             cv2.circle(image, center_out, 1, (255, 0, 0), 1)
#             cv2.circle(image, center_labels, 1, (0, 255, 0), 1)
#             plt.imshow(image)
#             plt.show()
#             #cv2.imshow("xd", image)
#             if i >= num_images:
                
#                 return

                
# visualize(model_pupil, dataloaders['test'], device, 5)

In [ ]:
# def evaluate(model, dataloaders, device, dataset_sizes):
#     model.eval()
#     correct = 0
#     with torch.no_grad():
#         for i, (inputs, labels) in enumerate(dataloaders["test"]):
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             outputs = model(inputs)
#             # print(outputs, labels)
#             diff = (labels[0][0] - outputs[0][0], labels[0][1] - outputs[0][1])
#             # print(diff)
#             dist = torch.sqrt(torch.pow(diff[0], 2) + torch.pow(diff[0], 2)) 
#             if dist < 2: 
#                 correct += 1
        
#         acc = correct / dataset_sizes['test']
#         print(acc)


In [ ]:
# pupil_trainer.train(device=device, num_epochs=5)

In [ ]:
# pupil_trainer.train(device=device, num_epochs=5)
pupil_trainer.model.load_state_dict(torch.load("models/weights/resnet50.pt"))


In [ ]:
def visualize_pupil(input_img, output_img):

    # outputs_soft = torch.softmax(output_img)
    # outputs_soft = np.transpose(outputs_sig.cpu().detach().numpy(), (1, 2, 0)).copy() 
    plt.imshow(input_img)
    plt.show()

    plt.imshow(output_img)
    plt.show()


In [ ]:

with torch.no_grad():
    pupil_trainer.model.eval()
    pupil_trainer.model = pupil_trainer.model.to(device)
    for i, (inputs, masks, opened) in enumerate(pupil_trainer.dataloaders['test']):
        
        inputs = inputs.to(device)
        outputs = pupil_trainer.model(inputs)

        image = np.transpose(inputs[0].cpu().numpy(), (1, 2, 0)).copy()
        outputs_sig = torch.sigmoid(outputs['out'][0])
        outputs_sig = np.transpose(outputs_sig.cpu().numpy(), (1, 2, 0)).copy()

        visualize_pupil(image, outputs_sig)
        ellipse = utils.fit_ellipse(outputs_sig)
        if ellipse:
            image = utils.draw_ellipse(image, ellipse)
            plt.imshow(image)
            plt.show()

        if i  == 5 :
            break
    

In [ ]:
#TODO 2 NN system
